## RUNNING MEDIAN

Compute the running median of a sequence of numbers. That is, given a stream of numbers, print out the median of the list so far on each new element.Recall that the median of an even-numbered list is the average of the two middle numbers. For example, given the sequence [2, 1, 5, 7, 2, 0, 5], your algorithm should print out:

- 2 (the median of a single number is that number) 
- 1.5 [2,1] --> the median is 3/2 = 1.5
- 2 [2,1,5] --> the median is 2 ... 
- 3.5 [2,1,5,7] --> the median is (2+5)/2 = 7/2 = 3.5
- 2 etc. etc. 

I guess the idea that first comes to mind is to sort the list. And then take the sorted list and find the median.. of course without using any build-in median command from python. To do this manually, it's probably about looking at the length of the list, and then finding out what the position is for the median... or doing an average if there are two middle numbers. OK let's try that and see where this problem goes.


In [12]:
t_list = [2, 1, 5, 7, 2, 0, 5]

# a small note on the difference between list.sort() and sorted( list )
print(sorted(t_list)) # returns an iterable list
print(t_list) # so you can see that t_list is not changed

[0, 1, 2, 2, 5, 5, 7]
[2, 1, 5, 7, 2, 0, 5]


In [14]:
# list.sort() doesn't return an iterable list, it changes the list
t_list = [2, 1, 5, 7, 2, 0, 5]

print(t_list)
print(t_list.sort()) # returns None
print(t_list) 

[2, 1, 5, 7, 2, 0, 5]
None
[0, 1, 2, 2, 5, 5, 7]


In [33]:
# OK let's find the median of this ... odd length list
t_list = [2, 1, 5, 7, 2, 0, 5]

# odd numbered list... so len(t_list) is 7 ... and we are looking for index position

index_median = len(t_list)/2 - 0.5 # small little hack bc i can't remember the round-down idea
index_median = int(index_median) 
median = sorted(t_list)[index_median]
print(median) 

2


In [47]:
# Let's look at the median of an even numbered list... should be straightforward 
t_list_even = [2,1,5,6] # the median here is (2+5)/2 = 3.5

index_median = len(t_list_even)/2 -1 # this gets us to 4/2 - 1 = index position 1 
index_median = int(index_median) 
median = (sorted(t_list_even)[index_median] + sorted(t_list_even)[index_median + 1])/2 
print(median) 

3.5


In [49]:
# Great, now combine the two ways .. 

def running_median(numeric_list):
    
    if len(numeric_list) % 2 == 0: # then this is even
        index_median = int(len(numeric_list)/2 - 1)
        median = (sorted(numeric_list)[index_median] + sorted(numeric_list)[index_median + 1])/2 
        return median 
    elif len(numeric_list) % 2 == 1: # then this is odd
        index_median = int(len(numeric_list)/2 - 0.5)
        median = sorted(numeric_list)[index_median] 
        return median 
    else: "wasn't expecting that" 


In [53]:
running_median([2,1,5,6]) # should return 3.5
running_median([2, 1, 5, 7, 2, 0, 5]) # should return 2 
running_median([5]) # should return 5

5

### Now that we've finished the simple version, now we need to figure out how to return this through a list.. and deliver sequential results as it works its way through the list
